In [2]:
pip install tensorflow==2.8.1 tensorflow-gpu==2.8.1 opencv-python mediapipe scikit-learn matplotlib

  Using cached tensorflow-2.8.1-cp39-cp39-win_amd64.whl (438.3 MB)
  Using cached tensorflow_gpu-2.8.1-cp39-cp39-win_amd64.whl (438.3 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached tensorflow_estimator-2.8.0-py2.py3-none-any.whl (462 kB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.2
    Uninstalling tensorboard-2.11.2:
      Successfully uninstalled tensorboard-2.11.2
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.5.1
    Uninstalling tensorflow-gpu-2.5.1:
     

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.1 requires keras<2.12,>=2.11.0, but you have keras 2.8.0 which is incompatible.
tensorflow-intel 2.11.1 requires tensorboard<2.12,>=2.11, but you have tensorboard 2.8.0 which is incompatible.
tensorflow-intel 2.11.1 requires tensorflow-estimator<2.12,>=2.11.0, but you have tensorflow-estimator 2.8.0 which is incompatible.


In [1]:
pip install tensorflow==2.11.1

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
#เปลี่ยน BGR เป็น RGB
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
#กำหนดlandmark
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
#customตัวlandmark
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,244,121), thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [101]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [222]:
DATA_PATH= os.path.join('MP_Data')

#actions = np.array(['สวัสดี', 'รอ', 'ขอบคุณ'])
#actions = np.array(['สวัสดี', 'รอ', 'กลับ', 'ขอบคุณ', 'ลง', 'ขึ้น', 'เฝ้า','คุย', 'ช่วย', 'เชื่อ', 'ฟัง', 'มอง', 'พา', 'ทักษิณ ชินวัตร', 'ขนม','แบก', 'อะไร', 'สมัคร', 'กระโดด', 'ยก', 'ชน', 'ผ่าน', 'แนะนำ', 'จำได้', 'ปลูก', 'รวม', 'ทำหาย', 'เจอ', 'หาย', 'พนักงานขาย', 'นักเขียนโปรแกรม', 'พ่อครัว', 'เจ้าหน้าที่ตำรวจ', 'เนื้อลูกแกะ', 'เนื้อหมู', 'คุณสบายดีไหม', 'ดูเหมือน', 'วาง', 'อยู่', 'ก', 'ข', 'ค', 'ง', 'จ', 'ช', 'อ'])
actions = np.array(['สวัสดี', 'ตก', 'รอ', 'กลับ', 'ขอบคุณ', 'ตัด', 'ลง', 'ขึ้น', 'เฝ้า','คุย', 'ช่วย', 'เชื่อ', 'ฟัง', 'มอง', 'พา', 'ชวน หลีกภัย', 'ทักษิณ ชินวัตร', 'ขนม', 'พิธา ลิ้มเจริญรัตน์', 'ศิริกัญญา ตันสกุล','แบก', 'อนุทิน ชาญวีรกูล', 'รังสิมันต์ โรม', 'พีระพันธุ์ สาลีรัฐวิภาค', 'คุณหญิงสุดารัตน์ เกยุราพันธุ์', 'พลเอกอนุพงษ์ เผ่าจินดา', 'สุวัจน์ ลิปตพัลลภ', 'กรณ์ จาติกวณิช', 'วราวุธ ศิลปะอาชา', 'พล.ต.อ.เสรีพิศุทธ์ เตมียเวสวราวุธ ศิลปะอาชา', 'ศักดิ์สยาม ชิดชอบ', 'ชาดา ไทยเศรษฐ์', 'สุชัชวีร์ สุวรรณสวัสดิ์', 'จุรินทร์ ลักษณวิศิษฏ์', 'ไตรรงค์ สุวรรณคีรี', 'พลเอกประยุทธ์ จันทร์โอชา', 'นฤมล ภิญโญสินวัฒน์', 'ธรรมนัส พรหมเผ่า', 'ชัยวุฒิ ธนาคมานุสรณ์', 'ไพบูลย์ นิติตะวัน', 'พลเอกประวิตร วงษ์สุวรรณ', 'พริษฐ์ วัชรสินธุ', 'ยิ่งลักษณ์ ชินวัตร', 'ณัฐวุฒิ ใสยเกื้อ', 'นพ.ชลน่าน ศรีแก้ว', 'เศรษฐา ทวีสิน', 'แพทองธาร ชินวัตร', 'อะไร', 'สมัคร', 'กระโดด', 'ยก', 'ชน', 'ผ่าน', 'แนะนำ', 'จำได้', 'ปลูก', 'รวม', 'ทำหาย', 'เจอ', 'หาย', 'พนักงานขาย', 'นักเขียนโปรแกรม', 'พ่อครัว', 'เจ้าหน้าที่ตำรวจ', 'เนื้อลูกแกะ', 'เนื้อหมู', 'คุณสบายดีไหม', 'ดูเหมือน', 'วาง', 'อยู่', 'ก', 'ข', 'ค', 'ฆ', 'ต', 'ถ', 'ง', 'จ', 'ฉ', 'ช', 'ซ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ท', 'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ'])

# 30 videos of data
no_sequences = 30

# videos are 30 frames length
sequence_length = 30

In [223]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [224]:
label_map = {label:num for num, label in enumerate(actions)}

In [225]:
label_map

{'สวัสดี': 0,
 'ตก': 1,
 'รอ': 2,
 'กลับ': 3,
 'ขอบคุณ': 4,
 'ตัด': 5,
 'ลง': 6,
 'ขึ้น': 7,
 'เฝ้า': 8,
 'คุย': 9,
 'ช่วย': 10,
 'เชื่อ': 11,
 'ฟัง': 12,
 'มอง': 13,
 'พา': 14,
 'ชวน หลีกภัย': 15,
 'ทักษิณ ชินวัตร': 16,
 'ขนม': 17,
 'พิธา ลิ้มเจริญรัตน์': 18,
 'ศิริกัญญา ตันสกุล': 19,
 'แบก': 20,
 'อนุทิน ชาญวีรกูล': 21,
 'รังสิมันต์ โรม': 22,
 'พีระพันธุ์ สาลีรัฐวิภาค': 23,
 'คุณหญิงสุดารัตน์ เกยุราพันธุ์': 24,
 'พลเอกอนุพงษ์ เผ่าจินดา': 25,
 'สุวัจน์ ลิปตพัลลภ': 26,
 'กรณ์ จาติกวณิช': 27,
 'วราวุธ ศิลปะอาชา': 28,
 'พล.ต.อ.เสรีพิศุทธ์ เตมียเวสวราวุธ ศิลปะอาชา': 29,
 'ศักดิ์สยาม ชิดชอบ': 30,
 'ชาดา ไทยเศรษฐ์': 31,
 'สุชัชวีร์ สุวรรณสวัสดิ์': 32,
 'จุรินทร์ ลักษณวิศิษฏ์': 33,
 'ไตรรงค์ สุวรรณคีรี': 34,
 'พลเอกประยุทธ์ จันทร์โอชา': 35,
 'นฤมล ภิญโญสินวัฒน์': 36,
 'ธรรมนัส พรหมเผ่า': 37,
 'ชัยวุฒิ ธนาคมานุสรณ์': 38,
 'ไพบูลย์ นิติตะวัน': 39,
 'พลเอกประวิตร วงษ์สุวรรณ': 40,
 'พริษฐ์ วัชรสินธุ': 41,
 'ยิ่งลักษณ์ ชินวัตร': 42,
 'ณัฐวุฒิ ใสยเกื้อ': 43,
 'นพ.ชลน่าน ศรีแก้ว': 44,
 'เศรษฐา ทวีส

In [226]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [227]:
len(sequences)

3270

In [228]:
X = np.array(sequences)

In [229]:
X.shape

(3270, 30, 1662)

In [51]:
# pose_keypoint_indices = range(0, 132)
# face_keypoint_indices = range(132, 1536)  # Assuming 21 hand keypoints after face keypoints
# hand_keypoint_indices = range(1536, 1662)  # Assuming the remaining keypoints are for pose

# # Remove face and pose keypoints from the data array
# X = np.concatenate((X[:, :, hand_keypoint_indices], X[:, :, pose_keypoint_indices]), axis=2)

# # Print the shape of the modified data array
# print(X.shape)

IndexError: index 1536 is out of bounds for axis 0 with size 46

In [230]:
y = to_categorical(labels).astype(int)

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [182]:
y_train.shape

(85, 3)

In [183]:
y_test.shape

(5, 3)

In [184]:
X_train.shape

(85, 30, 1662)

In [185]:
X_test.shape

(5, 30, 1662)

In [186]:
X[0][0].shape

# Why is the format of this input 30 * 1662?

(1662,)

In [187]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [188]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# model_alternative = Sequential()
# model_alternative.add(LSTM(256, return_sequences=False, activation='relu', input_shape=(30, 1662), dropout=0.2))
# model_alternative.add(Dense(3, activation='softmax')) #cheating on size
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [189]:
learning_rate = 0.00001

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [190]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_17 (LSTM)              (None, 128)               98816     
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 dense_20 (Dense)            (None, 3)                 195       
                                                                 
Total params: 549,379
Trainable params: 549,379
Non-trainable params: 0
_________________________________________________________________


In [191]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback],validation_data=(X_test, y_test), batch_size=32)

Epoch 1/100
3/3 [==============================] - 4s 300ms/step - loss: 1.1353 - categorical_accuracy: 0.3176 - val_loss: 1.0666 - val_categorical_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 0s 96ms/step - loss: 1.1261 - categorical_accuracy: 0.3176 - val_loss: 1.0632 - val_categorical_accuracy: 0.6000
Epoch 3/100
3/3 [==============================] - 0s 94ms/step - loss: 1.1192 - categorical_accuracy: 0.3176 - val_loss: 1.0585 - val_categorical_accuracy: 0.6000
Epoch 4/100
3/3 [==============================] - 0s 100ms/step - loss: 1.1123 - categorical_accuracy: 0.3176 - val_loss: 1.0543 - val_categorical_accuracy: 0.6000
Epoch 5/100
3/3 [==============================] - 0s 102ms/step - loss: 1.1067 - categorical_accuracy: 0.3176 - val_loss: 1.0525 - val_categorical_accuracy: 0.6000
Epoch 6/100
3/3 [==============================] - 0s 91ms/step - loss: 1.1026 - categorical_accuracy: 0.3176 - val_loss: 1.0515 - val_categorical_accuracy: 0.6000
Epoch 7/100
3

3/3 [==============================] - 0s 140ms/step - loss: 0.4782 - categorical_accuracy: 0.6706 - val_loss: 0.5812 - val_categorical_accuracy: 0.4000
Epoch 51/100
3/3 [==============================] - 0s 139ms/step - loss: 0.4697 - categorical_accuracy: 0.6706 - val_loss: 0.5820 - val_categorical_accuracy: 0.4000
Epoch 52/100
3/3 [==============================] - 0s 137ms/step - loss: 0.4634 - categorical_accuracy: 0.6706 - val_loss: 0.5675 - val_categorical_accuracy: 0.4000
Epoch 53/100
3/3 [==============================] - 0s 126ms/step - loss: 0.4521 - categorical_accuracy: 0.6824 - val_loss: 0.5466 - val_categorical_accuracy: 0.4000
Epoch 54/100
3/3 [==============================] - 0s 146ms/step - loss: 0.4441 - categorical_accuracy: 0.7294 - val_loss: 0.5314 - val_categorical_accuracy: 0.6000
Epoch 55/100
3/3 [==============================] - 1s 171ms/step - loss: 0.4368 - categorical_accuracy: 0.8000 - val_loss: 0.5229 - val_categorical_accuracy: 0.8000
Epoch 56/100
3/3 

3/3 [==============================] - 0s 123ms/step - loss: 0.0260 - categorical_accuracy: 0.9882 - val_loss: 0.0042 - val_categorical_accuracy: 1.0000
Epoch 100/100
3/3 [==============================] - 0s 115ms/step - loss: 0.0436 - categorical_accuracy: 0.9882 - val_loss: 0.0050 - val_categorical_accuracy: 1.0000


In [232]:
yhat = model.predict(X_test)

In [233]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [177]:
from sklearn.metrics import accuracy_score

In [234]:
accuracy_score(ytrue, yhat)

0.975609756097561

In [236]:
from sklearn.metrics import ConfusionMatrixDisplay,multilabel_confusion_matrix, classification_report, confusion_matrix
# cm = multilabel_confusion_matrix(ytrue, yhat)
cm = confusion_matrix(ytrue, yhat)
cm

array([[1, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [137]:
print(classification_report(ytrue, yhat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         5

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9



In [238]:
plt.rcParams['figure.figsize'] = (80, 80)
plt.rcParams['figure.dpi'] = 100

fig, ax = plt.subplots()

disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot(ax=ax)

# Increase font size of tick labels
ax.tick_params(axis='both', labelsize=30)

# Increase font size of colorbar tick labels
cbar = disp.im_.colorbar
cbar.ax.tick_params(labelsize=30)


for i in range(disp.confusion_matrix.shape[0]):
    for j in range(disp.confusion_matrix.shape[1]):
        plt.gca().text(j, i, disp.confusion_matrix[i, j], ha="center", va="center", color="white", fontsize=30)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(0.03, 0.03))
fig, ax = plt.subplots()

disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot(ax=ax)

plt.tight_layout()
plt.show()

In [199]:
model.save('action6.h5')


In [208]:
del model

In [209]:
from tensorflow.keras.models import load_model

model = load_model('action2.h5')

# ใช้จริง

In [70]:
from PIL import ImageFont, Image, ImageDraw

In [71]:
from scipy import stats

In [126]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    return output_frame

In [55]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'prob_viz' is not defined

<Figure size 1800x1800 with 0 Axes>

In [194]:
sequence = []
sentence = []
predictions = []
threshold = 0.80

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)

        fontpath = "./angsana.ttc"
        font = ImageFont.truetype(fontpath, 30)
        img_pil = Image.fromarray(image)
        draw = ImageDraw.Draw(img_pil)
        draw.text((30, 0),  ' '.join(sentence), font = font, fill = (255, 255, 255, 255))
        image = np.array(img_pil)
#         cv2.putText(image, ' '.join(sentence), (3,30), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

รอ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
รอ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
รอ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
รอ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
รอ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
รอ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ขอบคุณ
<class 'mediapipe.python.solution_base.SolutionOutputs'>


สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution

สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สวัสดี
<class 'mediapipe.python.solution

In [19]:
print("Length of actions:", len(actions))
print("Index used:", np.argmax(res))
print("Length of sentence:", len(sentence))
print("Last element of sentence:", sentence[0])

Length of actions: 3
Index used: 2
Length of sentence: 0


IndexError: list index out of range